<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Embeddings/word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from IPython.display import clear_output

In [23]:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
clear_output()

In [159]:
import bokeh
import bokeh.models as bm
import bokeh.plotting as pl
import gensim
import gensim.downloader as api
import nltk
import numpy as np
import pandas as pd

from bokeh.io import output_notebook
from gensim.models import Word2Vec
from nltk.tokenize import WordPunctTokenizer as WPT
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

## Bag of words

In [142]:
def splitme(doc):
  return [txt.lower().split() for txt in doc]


In [156]:
docs = ["I love pizza", "I like sushi", "I like pizza", "I enjoy beef"]
tk_docs = splitme(docs)
voc = list(set([word for doc in tk_docs for word in doc]))

wv = []
for d in tk_docs:
  vector = [d.count(word) for word in voc]
  wv.append(vector)

for i, word in enumerate(docs):
  #print(f"doc {i+1}:, {wv[i]}")
  print(f'doc{i+1}: {word} - {wv[i]}')

doc1: I love pizza - [0, 1, 0, 1, 0, 0, 1]
doc2: I like sushi - [0, 0, 0, 0, 1, 1, 1]
doc3: I like pizza - [0, 0, 0, 1, 1, 0, 1]
doc4: I enjoy beef - [1, 0, 1, 0, 0, 0, 1]


In [157]:
df = pd.DataFrame(wv, columns = voc)
df

,enjoy,love,beef,pizza,like,sushi,i
0,0,1,0,1,0,0,1
1,0,0,0,0,1,1,1
2,0,0,0,1,1,0,1
3,1,0,1,0,0,0,1


## Load data

In [10]:
data = list(open("quora.txt"))
data[666]

'How damning are the recent DNC leaks that show internal collusion?\n'

## Tokenize data

In [16]:
tokenizer = WPT()
print(tokenizer.tokenize(data[1]))

['What', 'are', 'some', 'ways', 'to', 'overcome', 'a', 'fast', 'food', 'addiction', '?']


In [17]:
new_data = [tokenizer.tokenize(x.lower()) for x in data]
new_data[3]

['what', 'are', 'zip', 'codes', 'in', 'the', 'bay', 'area', '?']

Bathe in tests

In [22]:
assert all(isinstance(row, list) for row in new_data)
assert all(all(isinstance(token, str) for token in row) for row in new_data)
is_latin = lambda token: all('a' <= x.lower() <= 'z' for x in token)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, new_data)))

In [28]:
vectorized = Word2Vec(new_data, window = 5, min_count=5, vector_size=32)

In [30]:
vectorized.wv.get_vector('vector')

array([ 2.6966121 , -1.7265937 ,  1.2603309 , -0.00504091,  1.4500982 ,
       -0.5708425 ,  0.2466997 ,  0.79414225,  0.02514288, -0.6594778 ,
        1.1856703 , -1.5489714 ,  1.2270495 , -0.15541814,  0.6565069 ,
        0.92180634, -2.7532423 ,  0.6869819 ,  0.9616499 ,  1.0173749 ,
       -2.0262227 ,  1.1631768 , -0.45039088, -0.77497727,  1.0542396 ,
       -0.76885736, -0.36369628,  0.60570604,  1.700349  ,  0.560057  ,
       -0.48080721,  1.7312859 ], dtype=float32)

In [51]:
vectorized.wv.vectors

array([[-0.36599022,  1.2630584 ,  0.7984064 , ..., -0.57422644,
        -0.47267944,  0.8781559 ],
       [-0.38964242,  4.1036725 ,  1.676159  , ...,  1.5856951 ,
        -1.1771752 ,  0.8522549 ],
       [-0.6092626 ,  3.915967  ,  0.7045298 , ..., -2.5133383 ,
        -1.049236  , -0.27340144],
       ...,
       [-0.0129451 , -0.10009976,  0.17413661, ..., -0.09894541,
        -0.01561986, -0.03023138],
       [-0.06435739, -0.05540132, -0.03262487, ..., -0.12932919,
        -0.11556447,  0.07254423],
       [-0.06711353, -0.08170432,  0.18834254, ..., -0.11174291,
        -0.06040575,  0.08545347]], dtype=float32)

In [60]:
list(zip(vectorized.wv.key_to_index.items()))[1030:1045]

[(('cards', 1030),),
 (('mathematics', 1031),),
 (('easily', 1032),),
 (('photo', 1033),),
 (('suitable', 1034),),
 (('front', 1035),),
 (('12th', 1036),),
 (('may', 1037),),
 (('early', 1038),),
 (('visiting', 1039),),
 (('baby', 1040),),
 (('treat', 1041),),
 (('building', 1042),),
 (('formula', 1043),),
 (('gets', 1044),)]

Cosine similary of the words is  
1. get two words in vector interpretation
2. Normalize them
3. Get their dot product
4. $cos\ similarity = \frac{word\_1\ @\ word\_2}{len(word\_1)\ *\  len(word\_2)}$
5. 1 means identical vectors, -1 - opposite, 0 -  no similarity\orthogonal directions (no shared context)

In [47]:
vectorized.wv.most_similar('faux')

[('pas', 0.983573853969574),
 ('extracurricular', 0.7581830620765686),
 ('cultural', 0.7327991127967834),
 ('traditions', 0.7185940146446228),
 ('appropriation', 0.7176169157028198),
 ('etiquette', 0.7153453826904297),
 ('pitfalls', 0.713373601436615),
 ('norms', 0.6962264180183411),
 ('brutality', 0.6933082342147827),
 ('criticisms', 0.6926724910736084)]

## Pre-trained model

In [62]:
api.info() #ruscorpora-300

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

In [63]:
model = api.load('glove-twitter-100') #weights ~400mb

[=================================================-] 99.8% 386.4/387.1MB downloaded


In [93]:
model.most_similar(["game", "multiplayer"], negative = ["fun"])

[('gameplay', 0.7124838829040527),
 ('crysis', 0.6885968446731567),
 ('dishonored', 0.683879554271698),
 ('battlefield', 0.6811695694923401),
 ('fps', 0.6733326315879822),
 ('rpg', 0.6518135070800781),
 ('starcraft', 0.6491492986679077),
 ('mmorpg', 0.642058253288269),
 ('simcity', 0.6411633491516113),
 ('dlc', 0.6395991444587708)]

In [117]:
words = model.index_to_key[:1000]
print(words)

['<user>', '.', ':', 'rt', ',', '<repeat>', '<hashtag>', '<number>', '<url>', '!', 'i', 'a', '"', 'the', '?', 'you', 'to', '(', '<allcaps>', '<elong>', ')', 'me', 'de', '<smile>', '！', 'que', 'and', '。', '-', 'my', 'no', '、', 'is', 'it', '…', 'in', 'n', 'for', '/', 'of', 'la', "'s", '*', 'do', "n't", 'that', 'on', 'y', "'", 'e', 'o', 'u', 'en', 'this', 'el', 'so', 'be', "'m", 'with', 'just', '>', 'your', '^', 'like', 'have', 'te', 'at', '？', 'love', 'se', 'are', '<', 'm', 'r', 'if', 'all', 'b', '・', 'not', 'but', 'we', 'es', 'ya', '&', 'follow', 'up', 'what', 'get', 'lol', 'un', '♥', 'lo', 'when', 'was', '“', '”', 'one', 'por', 'si', 'out', '_', 'mi', 'can', '<sadface>', 'من', '♡', '´', 'he', 'con', 'they', 'now', 'go', '،', 'para', 'los', 'know', 'haha', 'good', 'tu', 'back', '~', 'about', 'new', ';', 'as', 'day', 'how', 'who', 'will', 'want', 'people', 'yo', 'eu', 'from', 'di', 'time', '<heart>', 's', 'aku', 'da', "'re", '<lolface>', 'una', 'got', 'las', 'more', 'x', 'she', 'today', 

### Get vectorized words

In [118]:
words_array = np.array([model[word] for word in words])

Bathe in tests

In [119]:
assert isinstance(words_array, np.ndarray)
assert words_array.shape == (len(words), 100)
assert np.isfinite(words_array).all()

## Use PCA and normalize. reduce to two dim

In [165]:
pca = PCA(n_components=2)
components = pca.fit_transform(words_array)

scaler = StandardScaler()
pca_normalized = scaler.fit_transform(components)

Bathe in tests

In [114]:
assert pca_normalized.shape == (len(words_array), 2), "there must be a 2d vector for each word"
assert max(abs(pca_normalized.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - pca_normalized.std(0))) < 1e-2, "points must have unit variance"

Draw. Straight copied snippet

In [166]:
output_notebook() #bokeh.io
def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

draw_vectors(pca_normalized[:, 0], pca_normalized[:, 1], token=words)

Figure(id='1366', ...)

## t-SNE (t-distributed Stochastic neighbour embedding)

In [163]:
tsne = TSNE(n_components=2)
tsne_result  = tsne.fit_transform(words_array)

In [164]:
draw_vectors(tsne_result[:, 0], tsne_result[:, 1], color='green', token=words)

Figure(id='1234', ...)